In [8]:
from IPython.display import Audio   # reproducir audio
import pandas as pd                 # leer .csv

# modificar el path de la carpeta con los datos
DATA_FOLDER = './'

# se abre .csv con informacion de los audios
df = pd.read_csv(DATA_FOLDER + 'labels.csv')

# ---ejemplo de lectura de audio---

# se selecciona un audio aleatorio
idx = 127
sample = df.iloc[idx]

# se muestra informacion del audio y se escucha
print(f'Clase: ' + str(sample['class']))
print(f'Partición: ' + str(sample['partition']))
Audio(DATA_FOLDER + sample['path'])

Clase: 1
Partición: train


In [9]:
import os
import shutil
import pandas as pd

def duplicate_folders_and_csv(original_folder, new_folder, csv_path, new_csv_path):
    # Copiar carpetas de audio
    if not os.path.exists(new_folder):
        shutil.copytree(original_folder, new_folder)
    
    # Copiar el archivo CSV
    shutil.copy(csv_path, new_csv_path)
    
    print(f"Carpetas duplicadas en {new_folder} y CSV duplicado en {new_csv_path}")

# Ejemplo de uso:
original_folder = "./"  # Ruta de la carpeta original con train, validation, test
new_folder = "./CREMA-D.2"  # Ruta donde se duplicarán las carpetas
csv_path = "./CREMA-D/labels.csv"  # Ruta del CSV original
new_csv_path = "./CREMA-D/label2.csv"  # Ruta donde se guardará el nuevo CSV

duplicate_folders_and_csv(original_folder, new_folder, csv_path, new_csv_path)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: './CREMA-D'

In [ ]:
import librosa
import numpy as np

def augment_audio(audio_path, output_path, augment_type='speed'):
    y, sr = librosa.load(audio_path)
    
    # Aumentación: Cambio de velocidad
    if augment_type == 'speed':
        y_aug = librosa.effects.time_stretch(y, rate=np.random.uniform(0.9, 1.1))
    
    # Aumentación: Cambio de tono
    elif augment_type == 'pitch':
        y_aug = librosa.effects.pitch_shift(y, sr, n_steps=np.random.randint(-2, 3))
    
    # Guardar el audio aumentado
    librosa.output.write_wav(output_path, y_aug, sr)

def augment_class(csv_path, class_to_augment, n_augments, new_folder, new_csv_path):
    df = pd.read_csv(csv_path)
    
    # Filtrar audios de la clase que buscamos aumentar
    class_df = df[df['class'] == class_to_augment]
    
    # Seleccionar audios al azar para aumentar
    audios_to_augment = class_df.sample(n=n_augments)
    
    # Crear nueva carpeta si no existe
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)
    
    # Para cada audio seleccionado
    for i, row in audios_to_augment.iterrows():
        audio_path = row['path']  # Ruta del audio original
        augment_type = np.random.choice(['speed', 'pitch'])  # Elegir una técnica al azar
        
        # Generar un nuevo nombre para el archivo aumentado
        new_audio_name = f"{row['speaker_id']}_aug_{i}.wav"
        new_audio_path = os.path.join(new_folder, new_audio_name)
        
        # Aplicar augmentación
        augment_audio(audio_path, new_audio_path, augment_type)
        
        # Crear una nueva fila en el CSV
        new_row = row.copy()
        new_row['path'] = new_audio_path
        df = df.append(new_row, ignore_index=True)
    
    # Guardar el CSV actualizado
    df.to_csv(new_csv_path, index=False)
    print(f"Data augmentation completada. CSV actualizado en {new_csv_path}")